# <font color='Crimson'><b>MISSING DATA MANAGEMENT</b></font>

In [ ]:
#Import packages:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
.

In [ ]:
#Import datasets:
df_dataset = pd.read_csv('01_Dataset.csv',sep=',',index_col=0)
df_dataset.head()

In [ ]:
#Count of missing values per episode:
df_dataset_clean = df_dataset.drop(df_dataset.columns[[0,4,5]], axis=1)
df_dataset_clean_T = df_dataset_clean.T
#print(df_dataset_clean_T.shape)
ep_missing = (df_dataset_clean_T.isna().sum() / df_dataset_clean_T.shape[0]) * 100
ep_missing_sorted = ep_missing.sort_values()
#print(ep_missing_sorted.shape)

print('Number of episodes with more than 50% of missing values:',ep_missing_sorted[ep_missing_sorted > 50].shape[0], 
      '(',round(ep_missing_sorted[ep_missing_sorted > 50].shape[0] / df_dataset_clean_T.shape[1]*100,2),'%)')
print('--------------------------------------------------------------')
print('Quartiles of the distribution of missing values per episode:\n',ep_missing_sorted.quantile(q=list(np.arange(0, 1.1, 0.1))))

In [ ]:
#Visualization of the distribution of missing values per episode:
ecdf = pd.Series(ep_missing_sorted).value_counts(normalize=True).sort_index().cumsum()

plt.plot(ecdf.index, ecdf)
plt.xlabel("% of missingness")
plt.title("Distribution function of the % of missing values per episode")

plt.axhline(y=0.5, color='r', linestyle='-.')
plt.axvline(x=ep_missing_sorted.quantile(0.5), color='r', linestyle='-.')
plt.axhline(y=1-ep_missing_sorted[ep_missing_sorted > 50].shape[0]/df_dataset_clean_T.shape[1], color='b', linestyle='-.')
plt.axvline(x=50, color='b', linestyle='-.')

plt.show()

The majority of missing values is in the laboratory features (because of their sequential nature). Together with clinicians we decided that we keep only the first value (if present) for each laboratory exams result between the endpoints: 0,-1,-2.

In [ ]:
#Only the first laboratory value between 0 and -2:
df_merged = pd.DataFrame(np.zeros((df_dataset.shape[0], 47)), columns=range(0,47))
columns_names = ["ID","SESSO","ETA","REPARTO","DATA_PRELIEVO","MICRO","CANDIDEMIA","MISTA","30gg","MULTIFOCALE",
              "N_SITI","RESP","URINARIO","DIGESTIVO","BASOFILI","EOSINOFILI","LINFOCITI","MONOCITI",
              "NEUTROFILI","EMATOCRITO","EMOGLOBINA","GLOBULI_B","GLOBULI_R","PIASTRINE","APTT",
              "INR","TEMPO_PROTROMB","ACIDO_URICO","ALP","ALT","AST","BILIRUBINA_DIR","BILIRUBINA_TOT",
              "CREATININA","GGT","LAD","UREA","EMO_GLICATA","GLUCOSIO","ALBUMINA","B_D_GLUCANO","PCR",
              "PROCALCITONINA","PROTEINE_TOT","ACIDO_LATTICO_ART","ACIDO_LATTICO_VEN","TRIGLICERIDI"]

df_merged.columns = columns_names
n_columns = df_merged.shape[1]
index_last_a = df_dataset.columns.get_loc("Colonizzazione Candida Digestivo")

for i in range(df_dataset.shape[0]):
    for j in range(n_columns):
        if j <= index_last_a:
            df_merged.iloc[i, j] = df_dataset.iloc[i, j]
        else:
            if not pd.isna(df_dataset.iloc[i, j]):
                df_merged.iloc[i, j] = df_dataset.iloc[i, j]
            else:
                if not pd.isna(df_dataset.iloc[i, j+33]):
                    df_merged.iloc[i, j] = df_dataset.iloc[i, j+33]
                else:
                    df_merged.iloc[i, j] = df_dataset.iloc[i, j+33*2]
                    
df_merged["SESSO"] = df_merged["SESSO"].astype('category')
df_merged["ETA"] = df_merged["ETA"].astype(int)
df_merged["REPARTO"] = df_merged["REPARTO"].astype('category')
df_merged["CANDIDEMIA"] = df_merged["CANDIDEMIA"].astype('category')
df_merged["MISTA"] = df_merged["MISTA"].astype('category')
df_merged["30gg"] = df_merged["30gg"].astype('category')
df_merged["MULTIFOCALE"] = df_merged["MULTIFOCALE"].astype('category')
df_merged["N_SITI"] = df_merged["N_SITI"].astype('category')
df_merged["RESP"] = df_merged["RESP"].astype('category')
df_merged["URINARIO"] = df_merged["URINARIO"].astype('category')
df_merged["DIGESTIVO"] = df_merged["DIGESTIVO"].astype('category')
#df_merged.info()
df_merged.head()

In [ ]:
#Count of missing values per episode in the new data set:
df_merged_clean = df_merged.drop(df_merged.columns[[0,4,5]], axis=1)
df_merged_clean_T = df_merged_clean.T
#print(df_merged_T.shape)
ep_missing_bis = (df_merged_clean_T.isna().sum() / df_merged_clean_T.shape[0]) * 100
ep_missing_bis_sorted = ep_missing_bis.sort_values()
#print(ep_missing_bis_sorted.shape)

print('Number of episodes with more than 50% of missing values:',ep_missing_bis_sorted[ep_missing_bis_sorted > 50].shape[0], 
      '(',round(ep_missing_bis_sorted[ep_missing_bis_sorted > 50].shape[0] / df_merged_clean_T.shape[1]*100,2),'%)')
print('--------------------------------------------------------------')
print('Quartiles of the distribution of missing values per episode:\n',ep_missing_bis_sorted.quantile(q=list(np.arange(0, 1.1, 0.1))))

In [ ]:
#Visualization of the (new) distribution of missing values per episode:
ecdf_bis = pd.Series(ep_missing_bis_sorted).value_counts(normalize=True).sort_index().cumsum()

plt.plot(ecdf_bis.index, ecdf_bis)
plt.xlabel("% of missingness")
plt.title("Distribution function of the % of missing values per episode")

plt.axhline(y=0.5, color='r', linestyle='-.')
plt.axvline(x=ep_missing_bis_sorted.quantile(0.5), color='r', linestyle='-.')
plt.axhline(y=1-ep_missing_bis_sorted[ep_missing_bis_sorted > 50].shape[0]/df_merged_clean_T.shape[1], color='b', linestyle='-.')
plt.axvline(x=50, color='b', linestyle='-.')

plt.show()

We repeat the same evaluation only on the candidemia episodes because we don't want to lose too many (since they are only 10% of the total episodes): we observe the rate of missing per episode by selecting only the candidemia episodes ($x\%$ out of 1636):

In [ ]:
#Add the % of missingness per episode to the data set:
df_missing = pd.concat([df_merged, pd.DataFrame(ep_missing_bis)], axis=1)
#Select candidemia episodes only:
df_candidemie = df_missing[df_missing['CANDIDEMIA']==1]
candidemia_columns = columns_names[:]
candidemia_columns.append('% missing')
df_candidemie.columns = candidemia_columns

print('Number of candidemia episodes with more than 50% of missing values:',df_candidemie['% missing'][df_candidemie['% missing'] > 50].shape[0], 
      '(',round(df_candidemie['% missing'][df_candidemie['% missing'] > 50].shape[0] / df_candidemie.shape[0]*100,2),'%)')
print('--------------------------------------------------------------')
print('Quartiles of the distribution of missing values per episode:\n',df_candidemie['% missing'].quantile(q=list(np.arange(0, 1.1, 0.1))))

In [ ]:
#Visualization of the distribution of missing values per candidemia episode:
ecdf_tris = pd.Series(df_candidemie['% missing']).value_counts(normalize=True).sort_index().cumsum()

plt.plot(ecdf_tris.index, ecdf_tris)
plt.xlabel("% of missingness")
plt.title("Distribution function of the % of missing values per candidemia episode")

plt.axhline(y=0.5, color='r', linestyle='-.')
plt.axvline(x=df_candidemie['% missing'].quantile(0.5), color='r', linestyle='-.')
plt.axhline(y=1-df_candidemie['% missing'][df_candidemie['% missing'] > 50].shape[0]/df_candidemie.shape[0], color='b', linestyle='-.')
plt.axvline(x=50, color='b', linestyle='-.')

plt.show()

#### <font color='indianred'><b>CHOICE 1: We consider 77.9% of candidemia episodes </b></font>

We apply the same criterion for the episodes of bacteraemia and keep the episodes (regardless of candidemia or bacteremia) with a share of missing less or equal to 50%:

In [ ]:
#Keep episodes (regardless of candidemia or bacteremia) with a % of missing less or equal to 50%:
df_missing.columns = candidemia_columns
df_choice1 = df_missing[df_missing['% missing'] <= 50]
df_complete1 = df_choice1.drop(df_choice1.columns[47], axis=1)
print('Number of episodes kept:',df_complete1.shape[0])
print('--------------------------------------------------------------')
df_complete1.head()

<b>Have the proportions of candidemias been maintained with respect to the initial dataset?</b>

In [ ]:
#Comparison between proportions of candidemias:
print('Number of candidemias in the ORIGINAL dataset: ', pd.value_counts(df_merged['CANDIDEMIA'])[1], 
      '(',round(pd.value_counts(df_merged['CANDIDEMIA'], normalize=True)[1]*100,2),'%)')
print('--------------------------------------------------------------')
print('Number of candidemias in the CHOICE1 dataset: ', pd.value_counts(df_complete1['CANDIDEMIA'])[1], 
      '(',round(pd.value_counts(df_complete1['CANDIDEMIA'], normalize=True)[1]*100,2),'%)')

In [ ]:
#Count of missing values per variable:
df_complete1_missing = df_complete1.isna().mean() * 100 
df_complete1_missing.sort_values(ascending=False, inplace=True)

#Figure S1
#df_complete1_bis = df_complete1.drop(df_complete1.columns[[0,4,5]], axis=1) 
#df_complete1_missing = df_complete1_bis.isna().mean() * 100 
#df_complete1_missing.sort_values(ascending=False, inplace=True)
#columns_names = ["Glycated hemoglobin","BDG",                            
#                 "Gastrointestinal colonization",
#                 "Venous lactate", "Arterial lactate","Triglycerides",
#                 "Respiratory colonization",
#                 "Multifocal Candida colonization","PCT",
#                 "Direct bilirubin","Urinary colonization",
#                 "Lymphocyte cells count","Monocyte cells count",
#                 "Basophil cells count","Eosinophil cells count",
#                 "AST","Candida colonization","Uric acid","Total proteins","LDH","Albumin","Urea",
#                 "aPTT","INR","Prothrombin time","CRP","ALP","Total bilirubin",
#                 "Glucose","GGT", "ALT","Neutrophil cells count","Creatinine",
#                 "Ward of stay","Hematocrit","Hemoglobin","Red cells count",
#                 "Platelets count","White cells count",
#                 "# of explored colonization sites","Mixed infection",
#                 "Candidemia","Age","Sex"]
#df_complete1_missing.index = columns_names

plt.bar(df_complete1_missing.index, df_complete1_missing.values, color="#80b1d3")
plt.xlabel("Variable")
plt.ylabel("% of Missing Values")
#plt.title("% of Missing Values for Each Variable")
plt.xticks(rotation=90,size=6)
plt.axhline(y=50, color='#e41a1c', linestyle='-.')
#nome_figura = '<insert_your_path>/FigureS1.pdf'
#plt.savefig(nome_figura,dpi = 300,bbox_inches='tight')
plt.show()

#### <font color='indianred'><b>CHOICE 2: Max missing level 50% per feature</b></font>

In [ ]:
#Discard variables with more than 50% missing values:
df_complete2 = df_complete1[df_complete1_missing[df_complete1_missing <= 50].index]
df_complete2 = df_complete2.reindex(columns=['ID','SESSO','ETA','REPARTO','DATA_PRELIEVO',
                          'MICRO','CANDIDEMIA','MISTA','30gg','N_SITI',
                          'BASOFILI','EOSINOFILI','LINFOCITI','MONOCITI',
                          'NEUTROFILI','EMATOCRITO','EMOGLOBINA',
                          'GLOBULI_B','GLOBULI_R','PIASTRINE','APTT',
                          'INR','TEMPO_PROTROMB','ACIDO_URICO','ALP',
                          'ALT','AST','BILIRUBINA_TOT','CREATININA','GGT',
                          'LAD','UREA','GLUCOSIO','ALBUMINA','PCR','PROTEINE_TOT'])

print('Variables with less or equal than 50% of missing values:', df_complete2.columns.values)
print('--------------------------------------------------------------')
print('Shape of the final dataset:', df_complete2.shape)

In [ ]:
#Descriptive statistics of laboratoty exams mantained:
print('Main characteristics of laboratory exams mantained in the final data set:')
df_complete2.iloc[:,10:37].describe()

#### <font color='indianred'><b>EXPORT DATA SETS</b></font>

In [ ]:
#Export data sets to csv file:
df_complete1.to_csv('02_1_Dataset.csv', sep=',') #all variables
df_complete2.to_csv('02_2_Dataset.csv', sep=',') #variables with <=50% missing data